In [13]:
from ROOT import *
import numpy as np
import math
from shapely.geometry import LineString


In [4]:
#ch1=TChain("MCParticleNTuple/Tracks")
#ch1.Add("~/MightyIT/MCtracks_oldfile_20ev.root")

#ch1=TChain("MCParticleNTuple/Tracks")
#ch1.Add("~/MightyIT/MCtracks_oldfile_nocuts_20ev.root")

ch1=TChain("MCParticleNTuple/Tracks")
ch1.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_20ev.root")

#ch1=TChain("MCParticleNTuple/Tracks")
#ch1.Add("~/MightyIT/MCtracks_MagUp_bs2phiphi_1p5e34_nocuts_20ev.root")


gROOT.ProcessLine(".x ~/lhcbStyle.C")
gStyle.SetPaintTextFormat("1.3f")



In [5]:
#function that finds intersection of velo and t track for a longtrack, on the bending plane
#outputs the z coordinate (zmag)

def findZmag(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1): #to ask: velo p?
    tyV=(yv1 - yv0)/(zv1 - zv0)
    txV=(xv1 - xv0)/(zv1 - zv0)
    tyT=(yt1 - yt0)/(zt1 - zt0)
    txT=(xt1 - xt0)/(zt1 - zt0)
    
    #extrapolate velo track from v1 to t0 with velo slope
    xV_zt0 = xv1 + txV*(zt0-zv1)
    
    #extrapolate t track from t0 to v1 with t track slope
    xT_zv1 = xt0 + txT*(zv1-zt0)
    
    velo_extr = LineString([(xv1,zv1), (xV_zt0,zt0)])
    t_extr = LineString([(xT_zv1,zv1), (xt0,zt0)])
    
    #todo: sometimes no intersection, but this is not optimized?
    if velo_extr.intersection(t_extr):
        zmag = velo_extr.intersection(t_extr).y #it is actually z
    else:
        zmag = -1
   
    return zmag

In [ ]:
#FIRST TWO STRATEGY

In [71]:
########## FINDING ZMAG FOR VELOVELO SEGMENTS ########

#initialise histo and list

Nbins_p=100
Nbins_zmag=100
zmagvsp=TH2F("zmagvsp","",Nbins_p,Nbins_zmag,0,5000,4000,6000)
list_zmag = []

#find zmag for many long tracks
nEvent = 0
for event in ch1:
    
    #selecting some long tracks for event in ch1 and cut for !=0 else division/0
    if (event.HitVeloZpos[1]-event.HitVeloZpos[0] != 0 and event.HitZpos[1]-event.HitZpos[0] != 0 
       and  0.<event.HitVeloZpos[0]<event.HitVeloZpos[1]<800.  and event.HitZpos[0]>7000. and event.HitZpos[1]>7000 and event.p > 5000):
        nEvent = nEvent+1
        
        xv0 = event.HitVeloXpos[0]; yv0 = event.HitVeloYpos[0]; zv0 = event.HitVeloZpos[0];
        xv1 = event.HitVeloXpos[1]; yv1 = event.HitVeloYpos[1]; zv1 = event.HitVeloZpos[1];
        xt0 = event.HitXpos[0]; yt0 = event.HitYpos[0]; zt0 = event.HitZpos[0];
        xt1 = event.HitXpos[1]; yt1 = event.HitYpos[1]; zt1 = event.HitZpos[1];
        vp = event.p
        
      
        zmag = findZmag(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1) 
        if zmag != -1:
            zmagvsp.Fill(vp, zmag)
            list_zmag.append(zmag)
        #cut at n good events (computational time issue)
        if nEvent > 1000:
            break

Warning in TROOT::Append>: Replacing existing TH1: zmagvsp (Potential memory leak).


In [72]:
#plot zmag as function of momentum

c5=TCanvas("c5","",1400,1000)
zmagvsp.Fit("pol0")
zmagvsp.Draw()
c5.SaveAs("dist_testing_zmag.png")





****************************************
Minimizer is Linear
Chi2                      =  7.35746e+07
NDf                       =          884
p0                        =      5097.49   +/-   2.14192     


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c5
Info in <TCanvas::Print>: png file dist_testing_zmag.png has been created


In [8]:
########## FINDING ZMAG FOR VELOUT SEGMENTS ########
#initialise histo and list

Nbins_p=100
Nbins_zmag=100
zmagvsp=TH2F("zmagvsp","",Nbins_p,Nbins_zmag,0,5000,4000,6000)
list_zmag = []

#find zmag for many long tracks
nEvent = 0
for event in ch1:
    
    if (event.HitVeloZpos[1]-event.HitVeloZpos[0] != 0 and event.HitZpos[1]-event.HitZpos[0] != 0
       and  0.<event.HitVeloZpos[0]<800. and event.HitUTZpos[0]>2000 and event.HitUTZpos[0]<3000 and event.HitZpos[0]>7000. and event.HitZpos[1]>7000 and event.p > 5000):
        nEvent = nEvent+1

        #use fist hit in velo and first hit in UT
        xv0=event.HitVeloXpos[0];  yv0 = event.HitVeloYpos[0]; zv0 = event.HitVeloZpos[0];
        xv1=event.HitUTXpos[0];  yv1 = event.HitUTYpos[0]; zv1 = event.HitUTZpos[0];
        xt0 = event.HitXpos[0]; yt0 = event.HitYpos[0]; zt0 = event.HitZpos[0];
        xt1 = event.HitXpos[1]; yt1 = event.HitYpos[1]; zt1 = event.HitZpos[1];
        vp = event.p
        
      
        zmag = findZmag(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1) 
        if zmag != -1:
            zmagvsp.Fill(vp, zmag)
            list_zmag.append(zmag)
        #cut at n good events (computational time issue)
        if nEvent > 10000:
            break

Warning in <TROOT::Append>: Replacing existing TH1: zmagvsp (Potential memory leak).


In [9]:
#mean and sigma for the distribution (since it is quite constant)
print(np.mean(list_zmag))
print(np.std(list_zmag))




5137.983945428907
90.95516989777349


In [10]:
#plot zmag as function of momentum

c5=TCanvas("c5","",1400,1000)
zmagvsp.Fit("pol0")
zmagvsp.Draw()
c5.SaveAs("dist_testing_zmag.png")


****************************************
Minimizer is Linear
Chi2                      =  5.82813e+08
NDf                       =         3622
p0                        =      5133.91   +/-   0.716552    


Info in <TCanvas::Print>: png file dist_testing_zmag.png has been created


In [13]:
########## FINDING ZMAG FOR UTUT SEGMENTS ########
#initialise histo and list

Nbins_p=100
Nbins_zmag=100
zmagvsp=TH2F("zmagvsp","",Nbins_p,Nbins_zmag,0,5000,4000,6000)
list_zmag = []

#find zmag for many long tracks
nEvent = 0
for event in ch1:
    
    if (event.HitUTZpos[1]-event.HitUTZpos[0] != 0 and event.HitZpos[1]-event.HitZpos[0] != 0
       and  (event.HitUTZpos[1]>event.HitUTZpos[0]>2000) and (event.HitUTZpos[0]<event.HitUTZpos[1]<3000) and event.HitZpos[1]>event.HitZpos[0]>7000. and event.p > 5000):
        nEvent = nEvent+1

        #use fist two hita in UT
        xu0=event.HitVeloXpos[0];  yu0 = event.HitVeloYpos[0]; zu0 = event.HitVeloZpos[0];
        xu1=event.HitUTXpos[0];  yu1 = event.HitUTYpos[0]; zu1 = event.HitUTZpos[0];
        xt0 = event.HitXpos[0]; yt0 = event.HitYpos[0]; zt0 = event.HitZpos[0];
        xt1 = event.HitXpos[1]; yt1 = event.HitYpos[1]; zt1 = event.HitZpos[1];
        vp = event.p
        
      
        zmag = findZmag(xu0, yu0, zu0, xu1, yu1, zu1, xt0, yt0, zt0, xt1, yt1, zt1) 
        if zmag != -1:
            zmagvsp.Fill(vp, zmag)
            list_zmag.append(zmag)
        #cut at n good events (computational time issue)
        if nEvent > 10000:
            break

Warning in <TROOT::Append>: Replacing existing TH1: zmagvsp (Potential memory leak).


In [14]:
#mean and sigma for the distribution (since it is quite constant)
print(np.mean(list_zmag))
print(np.std(list_zmag))


5138.572882197544
91.12822517096782


In [15]:
#plot zmag as function of momentum

c5=TCanvas("c5","",1400,1000)
zmagvsp.Fit("pol0")
zmagvsp.Draw()
c5.SaveAs("dist_testing_zmag.png")


****************************************
Minimizer is Linear
Chi2                      =  5.17841e+08
NDf                       =         3562
p0                        =       5134.5   +/-   0.746421    


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c5
Info in <TCanvas::Print>: png file dist_testing_zmag.png has been created


In [ ]:
#FIRST LAST STRATEGY

In [6]:
def removezeros(listwithzerosX,listwithzerosY,listwithzerosZ):
    listwithoutzerosX = []
    listwithoutzerosY = []
    listwithoutzerosZ = []
    for item in range(0,len(listwithzerosZ)):
        if listwithzerosZ[item] != 0:
            listwithoutzerosX.append(listwithzerosX[item])
            listwithoutzerosY.append(listwithzerosY[item])
            listwithoutzerosZ.append(listwithzerosZ[item])
    return listwithoutzerosX,listwithoutzerosY,listwithoutzerosZ

In [7]:
def whichconfig(uzlist,vzlist):
    isassigned = False
    if len(uzlist)>0 and len(vzlist)>0:
        uz0 = uzlist[0]; vz0 = vzlist[len(vzlist)-1]
        if (uz0-vz0)!=0 and 0<vz0<800 and 2000<uz0<3000:
            isassigned=True
            return 'VeloUT' #best because most precise slope (long)
    elif len(uzlist)>1:
        uz0 = uzlist[0]; uz1 = uzlist[len(uzlist)-1]
        if (uz1-uz0)!= 0 and 2000<uz0<uz1<3000:
            isassigned=True
            return 'UTUT' #better than velovelo because hits are closer -> error propagated less
    elif len(vzlist)>1:
        vz0 = vzlist[0]; vz1 = vzlist[len(vzlist)-1]
        if (vz1-vz0)!=0 and 0<vz0<vz1<800:
            isassigned=True
            return 'VeloVelo'
    if isassigned == False:
        return -1

In [40]:

Nbins_p=100
Nbins_zmag=100
zmagvsp=TH2F("zmagvsp","",Nbins_p,Nbins_zmag,0,5000,4000,6000)
list_zmag = []

nEvent = 0
for event in ch1:
    if event.p > 5000:          
                velolistsxyz = removezeros(event.HitVeloXpos,event.HitVeloYpos,event.HitVeloZpos)
                velolistsx = velolistsxyz[0];velolistsy = velolistsxyz[1];velolistsz = velolistsxyz[2]

                utlistsxyz = removezeros(event.HitUTXpos,event.HitUTYpos,event.HitUTZpos)
                utlistsx = utlistsxyz[0];utlistsy = utlistsxyz[1];utlistsz = utlistsxyz[2]

                tlistsxyz = removezeros(event.HitXpos,event.HitYpos,event.HitZpos)
                tlistsx = tlistsxyz[0];tlistsy = tlistsxyz[1];tlistsz = tlistsxyz[2]


                eventconfig = whichconfig(utlistsz,velolistsz)
                #if eventconfig == 'VeloUT':
                    #xv0=velolistsx[0];  yv0 = velolistsy[0]; zv0 = velolistsz[0];
                    #xv1=utlistsx[len(utlistsz)-1];  yv1 = utlistsy[len(utlistsz)-1]; zv1 = utlistsz[len(utlistsz)-1];
                    #nEvent+=1
                if eventconfig == 'UTUT':
                    xv0=utlistsx[0];  yv0 = utlistsy[0]; zv0 = utlistsz[0];
                    xv1=utlistsx[len(utlistsz)-1];  yv1 = utlistsy[len(utlistsz)-1]; zv1 = utlistsz[len(utlistsz)-1];
                #if eventconfig == 'VeloVelo':
                    #xv0=velolistsx[0];  yv0 = velolistsy[0]; zv0 = velolistsz[0];
                    #xv1=velolistsx[len(velolistsz)-1];  yv1 = velolistsy[len(velolistsz)-1]; zv1 = velolistsz[len(velolistsz)-1];
                    #nEvent+=1
                

                    #selecting some long tracks for event in ch1 and cut for !=0 else division/0
                    if len(tlistsz)>1:
                        if (tlistsz[len(tlistsz)-1]>tlistsz[0]>7000 and tlistsz[len(tlistsz)-1]-tlistsz[0] != 0):
                            nEvent = nEvent+1
                            
                            xt0=tlistsx[0];  yt0 = tlistsy[0]; zt0 = tlistsz[0];
                            xt1=tlistsx[len(tlistsz)-1];  yt1 = tlistsy[len(tlistsz)-1]; zt1 = tlistsz[len(tlistsz)-1];
                            vp = event.p 

                            zmag = findZmag(xv0, yv0, zv0, xv1, yv1, zv1, xt0, yt0, zt0, xt1, yt1, zt1) 
                            if zmag != -1:
                                zmagvsp.Fill(vp, zmag)
                                list_zmag.append(zmag)
                            #cut at n good events (computational time issue)
                            if nEvent > 10000:
                                break
                

Warning in <TROOT::Append>: Replacing existing TH1: zmagvsp (Potential memory leak).


In [37]:
#velout
print(np.mean(list_zmag))
print(np.std(list_zmag))


5242.304049032313
98.55564206210029


In [41]:
#utut
print(np.mean(list_zmag))
print(np.std(list_zmag))


5290.142384112723
51.96271035348571


In [31]:
#velovelo
print(np.mean(list_zmag))
print(np.std(list_zmag))


5231.848068069948
126.42797052406657
